In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, basic
from lenskit import topn

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline

In [3]:
#ratings = pd.read_csv('/Users/denisehansen/Desktop/ITU/Thesis/Fair-Recommendations/ml-latest-small/ratings.1.csv', sep=',',
                      #names=['user', 'item', 'rating', 'timestamp'], header=0)
    
ratings = pd.read_csv('/Users/josse/Git-kode projects/Speciale/Fair-Recommendations/ml-latest-small/ratings.1.csv', sep=',',
                      names=['user', 'item', 'rating', 'timestamp'], header=0)
ratings.head()

,user,item,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
algo_ii = knn.ItemItem(20)
algo_als = als.BiasedMF(50)

In [5]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [6]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 2, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs.append(eval('ALS', algo_als, train, test))

test _reccomend
test _reccomend
test _reccomend
test _reccomend


In [7]:
all_recs = pd.concat(all_recs, ignore_index=True)
all_recs.head()

,item,score,user,rank,Algorithm
0,141,3.5,1,1,ItemItem
1,296,3.5,1,2,ItemItem
2,349,3.5,1,3,ItemItem
3,1968,3.5,1,4,ItemItem
4,1961,3.5,1,5,ItemItem


In [8]:
test_data = pd.concat(test_data, ignore_index=True)

In [9]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
results = rla.compute(all_recs, test_data)
results.head()

gcols
['user', 'Algorithm']
ti_cols
['user', 'item']
                ndcg
user Algorithm      
1    ALS         NaN
     ItemItem    NaN
2    ALS         NaN
     ItemItem    NaN
3    ALS         NaN
res.index:  MultiIndex(levels=[[1, 2, 3, 4, 5], ['ALS', 'ItemItem']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
           names=['user', 'Algorithm'])
true_key:  (1,)
Recs:
      item     score  user  rank Algorithm
60    2023  3.848754     1     1       ALS
61     858  3.610706     1     2       ALS
62    6711  3.547907     1     3       ALS
63    1221  3.547907     1     4       ALS
64    1197  3.423307     1     5       ALS
65    4963  3.397484     1     6       ALS
66    5816  3.397484     1     7       ALS
67    1193  3.397484     1     8       ALS
68     318  3.367344     1     9       ALS
69    2959  3.367344     1    10       ALS
70    3949  3.367344     1    11       ALS
71    2918  3.339544     1    12       ALS
72     903  3.339544     

C:\Users\josse\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


ndcg
user Algorithm          
1    ALS        0.000000
     ItemItem   0.000000
2    ALS        0.067314
     ItemItem   0.000000
3    ALS        0.063541